# Preprocessing Script


In [1]:
import numpy as np
import json
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
import plotly.io as pio
import plotly.express as px
print(pio.renderers)
pio.renderers.default = "notebook"

Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery']



### Formatting the the given csv

In [2]:
# path = "./data/result.csv"

# df = pd.read_csv(path, delimiter= ";" ,encoding = "ISO-8859-1")
# print("Column  Type")
# for col in df.columns:
#     print(col, type(df[col][0]))
#     if isinstance(df[col][0],str):
#         df[col] = df[col].apply(lambda s: " ".join(s.split()))

# print(type(df["EventTime"][0]))

# def changeDate(d):
#     d = d.replace(".000000000","")
#     d = d.replace("/","-")
#     return parse(d)
# df["EventTime"] = df["EventTime"].apply(changeDate)


# df.to_csv("./data/raw-alarms-test.csv", index=False)
# df.info()

### Reading csv from the formatted csv

In [3]:
path = "./data/raw-alarms-data.csv" # currently 1 month
cols = ["MachineName","SourceName","EventTime", "Message","Quality","Condition","Mask","NewState","Status"]
df = pd.read_csv(path, usecols=cols)
print("Column  Type")
for col in df.columns:
    print(col, type(df[col][0]))
df["EventTime"] = df["EventTime"].apply(lambda d: parse(d))
df.info()

Column  Type
MachineName <class 'str'>
SourceName <class 'str'>
EventTime <class 'str'>
Message <class 'str'>
Quality <class 'numpy.int64'>
Condition <class 'str'>
Mask <class 'numpy.int64'>
NewState <class 'numpy.int64'>
Status <class 'numpy.int64'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606532 entries, 0 to 606531
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   MachineName  606532 non-null  object        
 1   SourceName   606532 non-null  object        
 2   EventTime    606532 non-null  datetime64[ns]
 3   Message      606532 non-null  object        
 4   Quality      606532 non-null  int64         
 5   Condition    606532 non-null  object        
 6   Mask         606532 non-null  int64         
 7   NewState     606532 non-null  int64         
 8   Status       606532 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 41.6+ MB


In [4]:
df.tail(10)

,MachineName,SourceName,EventTime,Message,Quality,Condition,Mask,NewState,Status
606522,PHD47B,47TI931A,2019-03-21 03:39:37,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606523,PHD47B,47TI931A,2019-03-21 03:39:54,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606524,PHD47B,47TI931A,2019-03-21 08:05:48,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606525,PHD47B,47TI931A,2019-03-21 08:08:08,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606526,PHD47B,47TI931A,2019-03-21 08:02:07,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606527,PHD47B,47TI931A,2019-03-21 08:05:43,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606528,PHD47B,47TI931A,2019-03-21 08:00:59,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606529,PHD47B,47TI931A,2019-03-21 08:00:17,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606530,PHD47B,47TI931A,2019-03-21 03:35:43,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
606531,PHD47B,48AI004,2019-03-21 03:35:30,48AI004 48S-1 H2S PV = 0.00 %H2S IOP,0,IOP,201,3,1


# Preprocessing of Alarm data

There are different types of alarm data but Excel file only contains "Process Alarm Data".

**Excel Columns Description**
* MachineName: It represent the unit (e.g.,Plant-47)
* SourceName: The names of the devices operating in a unit (e.g., controller, transmitter, and calculator block names)
    1. Five types of controllers: TIC, PIC, FIC, LIC, and AIC.
    2. Three types of transmitters: TI, FI, LI
    3. Calculator block names ?

* EventTime: The date when an alarm is appeared. It also represent when it recovered right?

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state. 

**Questions:**
1. Before alarm data analysis, if ALM, NR and RECOVER messages are seen, these rows must be deleted. Why?
2. Alarm-off 
3. shelving of alarms
4. Can alarm cannot trigger twice? 
5. 

In [5]:
sources = {}
source_conditions = {}
conditions = {} # Vel, IOP ALM, NR etc
key = ""
temp_dict = None
d = None
conditions["Recover"] = 0
for i in range(0, len(df["EventTime"])):
    key = df["SourceName"][i]
    temp_dict = {"Quality":df["Quality"][i], "Mask":df["Mask"][i], "NewState":df["NewState"][i], "Status": df["Status"][i] ,"SourceName": key,"EventTime": df["EventTime"][i], "Message":df["Message"][i],"Condition": df["Condition"][i]}
    
    if sources.get(key) != None:
        sources[key].append(temp_dict)
        
        #source conditions
        d =  source_conditions[key]
        if d.get(temp_dict["Condition"]) == None:
            d[temp_dict["Condition"]] = 0    
        d[temp_dict["Condition"]]  += 1
        d["Total"] += 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        # conditions           
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] += 1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1
    else:
        sources[key] = []
        source_conditions[key] = {}
        sources[key].append(temp_dict)
        d = source_conditions[key]
        d[temp_dict["Condition"]] = 1
        d["Total"] = 1
        
        if d.get("Recover") == None:
            d["Recover"] = 0
        
        c = temp_dict["Condition"]
        if conditions.get(c) == None:  
            conditions[c] = 0
        conditions[c] +=1
        
        if temp_dict["Message"].find("Recover") != -1:
            conditions["Recover"] += 1
            d["Recover"] +=1
        if temp_dict["Message"].strip().find("NR") != -1:
#             print(temp_dict["Message"])
            d["Recover"] +=1
            conditions["Recover"] += 1


# sorting the source conditions dictionary based on the number of alarms related to each sensor
source_conditions = {k:v for k,v in sorted(source_conditions.items(), key=lambda item: item[1]["Total"],reverse=True)}
conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}


### Total number of disctinct SourceNames in the excel sheet.

In [6]:

print("Total Number of Sourcenames: {s}".format(s=len(sources.keys())))

print("SourceName")
for s in sources.keys():
    print(s)


Total Number of Sourcenames: 341
SourceName
48FIX2040
47TI931A
48TIC2027
47PI1734
48PI2025
47TI1713
47LIC3103
47LIC3408
48XL002-ANN
SIO-18XA001
47TI2111A
47TIC3520
47TI2111B
47FLI-039
47LIC3409
48AI004
48AAHH004-ANN
47LI024
47TI1512B
47TI1512A
47TI1512ABC
48FI2003A
48FALL2003-ANN
47TI2112B
47TI002C
47FIC1522A
47TXI1592
47LIC005
47TI1512C
48FFIC2002
48XL012-ANN
47UA2500-ANN
47PIC027-ANN
47PIC2028
47TI870D
47PDI003
47XL016-ANN
47XL015-ANN
47FSL018-ANN
47LI003
47FIC1520
47TI003B
47PIC1505
47TI2077
47TI2076
47TI2075
47TI2073
19LI105A
47TIC2078
47TAL003AB-ANN
47TI003A
48LI010
48LIC2003
47FLI-042
47PIC1760
47TI1508
47TIC259
47TI1705
47PIC2033A
47TI3531
19ZAH119A-ANN
48FI2021B
47TI3519
19PI119A
19ZAL119A-ANN
48LIC009
47TI2072
47TI1516
47TI1506
47LI026A
47LI026B
47TI414Q
47FD-032H-ANN
47FIC1523
47LIC1523
47TI416
47LI026C
47EM306B-ANN
47PI1710
48FI012
47LI3104A
47PDI3104
47XL017G-ANN
47TXI1594
47PI2031A
47HCD-047H-ANN
47EM306A-ANN
47TI017
19TI104
47XL017B-ANN
47FIC033
47PI2031ABC
47TI2003
47XL3

### How many times each SourceName has been triggered (activation + Deactivation)?

In [7]:

total  = 0
i =0
for key in source_conditions.keys():
    s1 = "Source: " + key + ", Total: {} | ".format(source_conditions[key]["Total"])
    s2 = ""
    total += source_conditions[key]["Total"]
    for k in source_conditions[key].keys():
        if k == "Total" or k == "Recover":
            continue
        s2 = s2 + k +" "+ str(source_conditions[key][k]) + ", "    
    s1 = s1 + s2
    print("{}. ".format(i),s1)
    i +=1
#     if i == 10:
#         break
print("\n\n Total Number of Entries in the excel:", total)
# assert(total== len(df["SourceName"]))

0.  Source: 47TI931A, Total: 318539 | IOP 318539, 
1.  Source: 48FIX2040, Total: 67470 | IOP- 67470, 
2.  Source: 48TIC2027, Total: 67469 | IOP- 67469, 
3.  Source: 47PI1734, Total: 53223 | IOP 43666, IOP- 6951, HI 572, HH 498, LO 739, LL 797, 
4.  Source: 48PI2025, Total: 13213 | IOP 13019, HH 64, HI 64, IOP- 66, 
5.  Source: 47TI003A, Total: 11117 | LTRP 10892, LLL 225, 
6.  Source: 47TAL003AB-ANN, Total: 11007 | ALM 11007, 
7.  Source: 47TI002C, Total: 9470 | LLL 9470, 
8.  Source: 47TI1713, Total: 9255 | IOP 8121, IOP- 1134, 
9.  Source: 19LI105A, Total: 8848 | VEL+ 702, VEL- 2092, LL 2501, LO 3547, IOP- 6, 
10.  Source: 47TI003B, Total: 7419 | LTRP 6830, LLL 589, 
11.  Source: 48AAHH004-ANN, Total: 3537 | ALM 3537, 
12.  Source: 48AI004, Total: 3530 | IOP 3530, 
13.  Source: 47FI1575, Total: 1299 | IOP 1299, 
14.  Source: 47LIC3103, Total: 1154 | LO 175, HI 798, LL 59, HH 122, 
15.  Source: 47LI003, Total: 1072 | HTRP 536, LTRP 525, HHH 11, 
16.  Source: 48XL002-ANN, Total: 940 | 

### Number of times each condition appear

In [8]:
plot_dict = {} 
plot_dict["Condition"] = [k for k in conditions.keys()]
plot_dict["Freq"] = [v for v in conditions.values()]
# print(x_axis)
# print(y_axis)

for k in conditions.keys():
#     if k == "Recover":
#         continue
    print(k,conditions[k])

# print("Recover", conditions["Recover"]) ### including "NR" message

# fig = px.bar(x=plot_dict["Condition"],y=plot_dict["Freq"])

# fig.show()

IOP 389530
Recover 300935
IOP- 143119
ALM 18563
LTRP 18483
LLL 11389
LO 7295
LL 4330
HI 4261
VEL- 2166
HHH 2128
HTRP 1330
HH 1266
CNF 810
VEL+ 809
ANS- 648
DV- 272
DV+ 130
MHI 3


### Displaying Messages of top SourceName alarms (oreder by EventTime)
Which message is related to which message (i.e., activation and deactivation) ?


In [9]:
# conditions = {k:v for k,v in sorted(conditions.items(), key=lambda item: item[1], reverse=True)}

i = 0
for sname in source_conditions.keys():
    
    print("=======Source {}: {}========".format(i,sname))
    single_source_alarms = sources[sname]
    single_source_alarms = [v for v in sorted(single_source_alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    
    j = 0
    for alarm in single_source_alarms:
        print("{},---,{},--- Message: {}".format(alarm["EventTime"],alarm["Condition"], alarm["Message"]))
        j += 1
        if j == 30:
            break
    i += 1
    if i == 10:
        break


=======Source 0: 47TI931A========
2019-03-06 13:19:13,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2019-03-06 13:19:17,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2019-03-06 13:19:33,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2019-03-06 13:19:35,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2019-03-06 13:20:22,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2019-03-06 13:20:24,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2019-03-06 13:20:28,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2019-03-06 13:20:30,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2019-03-06 13:20:49,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP Recover
2019-03-06 13:20:51,---,IOP,--- Message: 47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP
2019-03-06 13:21:03,---,IO

2019-03-06 13:22:43,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:31:34,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP Recover
2019-03-06 13:31:35,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:34:14,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP Recover
2019-03-06 13:34:15,---,IOP-,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP-
2019-03-06 13:34:16,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:34:16,---,IOP-,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP- Recover
2019-03-06 13:34:50,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP Recover
2019-03-06 13:34:51,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:39:59,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP Recover
2019-03-06 13:40:00,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:43:33,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP Recover
2019-03-06 13:43:34,---,IOP,--- Message: 47PI1734 PV = 0.62 Kg/cm2 IOP
2019-03-06 13:43:52,---,I

### **Find the duration of activation of each Alarm i.e., duration =  DeactivationTime - ActivationTime**

Compute mean, median, mode, on the duration...

**Deactivation of an Alarm:**
* Recover : The alram is deactivated.
* IOP  : Communication between the DCS and field is cut off (alarm activated).
* IOP + Recover: communication problem is solved and the alarm is deactivated. 
* ALM : It represent alarm is activated
* NR : The alarm is returned to normal state.


In [10]:
from datetime import timedelta
import statistics
from collections import Counter
# c = Counter([1,1,2,2,3])
# c.most_common(1)

def getDurationAndStats(alarms):
    """Returns list of dict and stats dictionary"""
#     queue = []
    conditions_queues = {}
    stats = {} 
    start_end = []
    d = None
    alarms = [v for v in sorted(alarms, key=lambda arg: arg["EventTime"], reverse = False) ]
    for alarm in alarms:
#         print(alarm["Message"])
        if conditions_queues.get(alarm["Condition"]) == None:
            conditions_queues[alarm["Condition"]] = []

        if alarm["Message"].find("Recover") == -1 and alarm["Message"].find("NR") == -1:
            conditions_queues[alarm["Condition"]].append(alarm) # enqueue the activation
        else:
            if len(conditions_queues[alarm["Condition"]])== 0:
#                 print("(", alarm["SourceName"],alarm["Message"], ")")
                continue
            d = conditions_queues[alarm["Condition"]].pop(0) # dequeue
            d = {k:v for k,v in d.items()}
            d["StartTime"] = d["EventTime"]
            d["EndTime"] = alarm["EventTime"]
            d["EndMessage"] = alarm["Message"]
            del d["EventTime"]
            
            start_end.append(d)
#     print("Remaining Elements in Queue", len(queue))
    temps = []
    for alarm in start_end:
        temps.append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
#     print(">>log ", len(start_end))
    if len(start_end) == 0:
        return None,None
    
    stats["timedeltas"] = [v for v in sorted(temps)]
    stats["mean"] =  round(statistics.mean(stats["timedeltas"]),3) 
    stats["median"] = statistics.median(stats["timedeltas"])
    stats["mode"] =  Counter(stats["timedeltas"]).most_common(1)#statistics.mode(stats["timedeltas"])
    stats["min"] = stats["timedeltas"][0]
    stats["max"] = stats["timedeltas"][-1]
    return (start_end, stats)

In [11]:
i = 0
for key in source_conditions.keys():
    d = source_conditions[key]  
    print("{}. Source: {}, # of activations= {}, # of deactivations = {}, Diff = {}".format(i,key, d["Total"]-d["Recover"], d["Recover"], (d["Total"]-d["Recover"])-d["Recover"]))
    i += 1
#     if i > 4:
#         break

print ("==================================================================================")
i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
#     if i > 10: 
#         break
      
    alarms = sources[sname]
    _, stats = getDurationAndStats(alarms)
    if stats == None:
        continue
    print("{}. source:{}, Total alarms:{} | min = {}, max = {}, median = {}, mode = {}, mean = {}".format(i,sname,len(stats["timedeltas"]) ,stats["min"], stats["max"], stats["median"], stats["mode"], stats["mean"]))
    i += 1
print("Note: All the stats are in secods")

0. Source: 47TI931A, # of activations= 159270, # of deactivations = 159269, Diff = 1
1. Source: 48FIX2040, # of activations= 33735, # of deactivations = 33735, Diff = 0
2. Source: 48TIC2027, # of activations= 33734, # of deactivations = 33735, Diff = -1
3. Source: 47PI1734, # of activations= 26613, # of deactivations = 26610, Diff = 3
4. Source: 48PI2025, # of activations= 6606, # of deactivations = 6607, Diff = -1
5. Source: 47TI003A, # of activations= 5559, # of deactivations = 5558, Diff = 1
6. Source: 47TAL003AB-ANN, # of activations= 5525, # of deactivations = 5482, Diff = 43
7. Source: 47TI002C, # of activations= 4742, # of deactivations = 4728, Diff = 14
8. Source: 47TI1713, # of activations= 4628, # of deactivations = 4627, Diff = 1
9. Source: 19LI105A, # of activations= 4423, # of deactivations = 4425, Diff = -2
10. Source: 47TI003B, # of activations= 3709, # of deactivations = 3710, Diff = -1
11. Source: 48AAHH004-ANN, # of activations= 1767, # of deactivations = 1770, Diff =

1. source:47TI931A, Total alarms:159266 | min = 0.0, max = 63763.0, median = 33.0, mode = [(28.0, 4746)], mean = 35.762
2. source:48FIX2040, Total alarms:33734 | min = 1.0, max = 327257.0, median = 15.0, mode = [(13.0, 1944)], mean = 57.328
3. source:48TIC2027, Total alarms:33734 | min = 1.0, max = 327257.0, median = 15.0, mode = [(13.0, 1914)], mean = 57.17
4. source:47PI1734, Total alarms:26606 | min = 0.0, max = 155319.0, median = 42.0, mode = [(6.0, 547)], mean = 314.339
5. source:48PI2025, Total alarms:6604 | min = 0.0, max = 84521.0, median = 40.0, mode = [(1.0, 263)], mean = 429.555
6. source:47TI003A, Total alarms:5556 | min = 0.0, max = 482009.0, median = 4.0, mode = [(2.0, 1128)], mean = 621.289
7. source:47TAL003AB-ANN, Total alarms:5479 | min = 0.0, max = 212359.0, median = 755.0, mode = [(3.0, 49)], mean = 10014.276
8. source:47TI002C, Total alarms:4727 | min = 1.0, max = 240946.0, median = 43.0, mode = [(5.0, 129)], mean = 2558.952
9. source:47TI1713, Total alarms:4625 | 

68. source:47PI2042, Total alarms:35 | min = 1.0, max = 20161.0, median = 3.0, mode = [(3.0, 12)], mean = 1145.314
69. source:48FI2021B, Total alarms:26 | min = 26595.0, max = 1160797.0, median = 633514.5, mode = [(26595.0, 1)], mean = 520073.5
70. source:47LI024, Total alarms:26 | min = 3.0, max = 955309.0, median = 243638.5, mode = [(3.0, 1)], mean = 301000.231
71. source:48LI011, Total alarms:26 | min = 409.0, max = 888806.0, median = 306595.5, mode = [(409.0, 1)], mean = 341878.808
72. source:47TXI1592, Total alarms:26 | min = 58.0, max = 474925.0, median = 319076.0, mode = [(58.0, 2)], mean = 303033.423
73. source:47TI1506, Total alarms:24 | min = 1210.0, max = 826729.0, median = 410801.0, mode = [(4329.0, 2)], mean = 387136.792
74. source:47TXI1594, Total alarms:24 | min = 2848.0, max = 506869.0, median = 306910.0, mode = [(2848.0, 1)], mean = 299024.083
75. source:47TI1705, Total alarms:20 | min = 57978.0, max = 819232.0, median = 451610.0, mode = [(57978.0, 1)], mean = 457786.7

# Exporintg results to csv including communication alarms (IOPs)


In [21]:
import calendar

csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta": []} 

csv["Year"]= []
csv["Month"] = []
csv["MonthDay"] = []
csv["WeekDay"] = []
csv["Time"]= []
csv["Hour"] = []
csv["Minute"] = []
d = None
i = 1
for sname in source_conditions.keys():
#     sname = "48TIC2026"
# Break to get top 10
#     if i > 10: 
#         break
#     i += 1    

#     sname = "48TIC2026"

    alarms = sources[sname]
    alarms,stats = getDurationAndStats(alarms)
    if alarms == None:
        continue

    for alarm in alarms:
        d = alarm["StartTime"]
        csv["SourceName"].append(alarm["SourceName"]) 
        csv["StartTime"].append(alarm["StartTime"]) 
        csv["EndTime"].append(alarm["EndTime"])
        csv["TimeDelta"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
        csv["Message"].append(alarm["Message"])
        csv["RecoveryMessage"].append(alarm["EndMessage"])
        csv["Quality"].append(alarm["Quality"])
        csv["Condition"].append(alarm["Condition"])
        csv["Mask"].append(alarm["Mask"])
        csv["NewState"].append(alarm["NewState"])
        csv["Status"].append(alarm["Status"])
        csv["Year"].append(d.year)
        csv["Month"].append(d.month)
        csv["MonthDay"].append(d.day)
        csv["WeekDay"].append(calendar.day_name[d.weekday()])
        csv["Time"].append(d.strftime("%H:%M"))
        csv["Hour"].append(d.hour)
        csv["Minute"].append(d.minute)


print("Total Entries :", len(csv["Status"]))
data = pd.DataFrame(csv)
data.to_csv("./data/stats-with-IOPs.csv", index = False)

Total Entries : 300884


# Exporintg results to csv withiout communication alarms (IOPs)

In [13]:

# csv = {"SourceName":[],"StartTime":[], "EndTime": [] , "Message" : [], "RecoveryMessage":[] ,"Quality": [],"Condition":[],"Mask":[],"NewState": [],"Status": [],"TimeDelta": []} 

# csv["Year"]= []
# csv["Month"] = []
# csv["MonthDay"] = []
# csv["WeekDay"] = []
# csv["Time"]= []
# csv["Hour"] = []
# csv["Minute"] = []
# d = None
# i = 1
# for sname in source_conditions.keys():

# # Break to get top 10
#     if i > 10: 
#         break
#     i += 1    

# #     sname = "48TIC2026"

#     alarms = sources[sname]
#     alarms,stats = getDurationAndStats(alarms)
#     if alarms == None:
#         continue

#     for alarm in alarms:
#         if alarm["Condition"] == "IOP" or alarm["Condition"] == "IOP-":
#             continue
#         d = alarm["StartTime"]
#         csv["SourceName"].append(alarm["SourceName"]) 
#         csv["StartTime"].append(alarm["StartTime"]) 
#         csv["EndTime"].append(alarm["EndTime"])
#         csv["TimeDelta"].append(timedelta.total_seconds (alarm["EndTime"]-alarm["StartTime"]))
#         csv["Message"].append(alarm["Message"])
#         csv["RecoveryMessage"].append(alarm["EndMessage"])
#         csv["Quality"].append(alarm["Quality"])
#         csv["Condition"].append(alarm["Condition"])
#         csv["Mask"].append(alarm["Mask"])
#         csv["NewState"].append(alarm["NewState"])
#         csv["Status"].append(alarm["Status"])
#         csv["Year"].append(d.year)
#         csv["Month"].append(d.month)
#         csv["MonthDay"].append(d.day)
#         csv["WeekDay"].append(calendar.day_name[d.weekday()])
#         csv["Time"].append(d.strftime("%H:%M"))
#         csv["Hour"].append(d.hour)
#         csv["Minute"].append(d.minute)


# print("Total Entries written in to CSV :", len(csv["Status"]))
# data = pd.DataFrame(csv)
# data.to_csv("./data/stats-without-IOPs.csv", index = False)

Total Entries written in to CSV : 21542
